# Solution - exercise sheet 3
### Due 18/11/2022, 14:00 

## The concept

The **trapezoid method** is one of the most popular numerical integration techniques.

To first degree ($n=0$), the trapezoid method is just the area under a trapezoid connecting the two end points of our function:

$
\begin{align}
n=0: \int_a^b f(x) dx \,\approx\, I_0 = \frac{1}{2} h_0 [f(a) + f(b)], \hspace{5mm}\mathrm{where}\;h_0 = (b - a) \hspace{5cm}(1)
\end{align}
$

This $I_0$ approximation gives a very crude estimate of the integral. To increase our precision, we can divide the region between $a$ and $b$ into $n+1$ segments, calculate the area within these segments, and add them up. The formula for the n-th degree of accuracy of this algorithm is:

$
\begin{align}
n>0: \int_a^b f(x) dx \,\approx\, I_n = \frac{1}{2} A_{n-1} + h_n\sum_{i=1,3.,,,}^{2^n - 1} f(a + i h_n), \hspace{5mm}\mathrm{where}\;h_n=\frac{1}{2}h_{n-1} \hspace{3.3cm}(2)
\end{align}
$

As you can see, the result of the integral using the $n$-th degree of accuracy of this agorithmis is given as a recurrence relation,  $I_n=I_n(I_{n-1})$.

As you calculate the algorithm for higher degree $n$, your approximation $I_n$ assimptotically approaches the result of the integral. The precision of your approximation can then be estimated by looking at the relative change between the last two steps:

$
\begin{align}
\varepsilon(n) = \frac{I_{n} - I_{n-1}}{I_n} \hspace{10cm}(3)
\end{align}
$

## The daunting-looking task

You'll be building a function `trapz(f,a,b, prec)` that calculates the definite integral of a function `f` between `a` and `b`, down to a precision of `prec`.

For this, `trapz` calculates approximations of the integral using the trapezoidal algorithm, starting from $n=0$ and successively increasing $n$ until $|\varepsilon(n)|<$`prec` and our precision is satisfied. 

Once the precision is satisfied at $n=n_\mathrm{final}$, our function returns $I_{n_\mathrm{final}}$, $n_\mathrm{final}$, and $\varepsilon(n_\mathrm{final})$.

## The step-by-step breakdown

#### 1. Write a function `h_n` that returns the value of $h_n(a,b)$.

- Tip 1: Check out the example of a recurrence function we worked out in the lecture. 
- Tip 2: Remember that $h_n$ is defined recursively for $n>0$ (eq. 2) and has a special definition for $n=0$ (see eq. 1)

In [68]:
def h_n(n, a, b):
    """ Recursively calculate the width of 2^n trapezoids.
    
    Args:
        n: number of intervals
        a: lower bound of integration
        b: upper bound of integration

    Returns: 
        The length of an interval
    """
    if n == 0:
        return b - a
    else:
        return h_n(n - 1, a, b) * 0.5

#### 2. Write a function `sum_terms` that calculates the sum in eq. (2).

- Tip 1: for a given `n` (passed to `sum_terms` as an argument), you must iterate over `i` from 1 to $2^n-1$ in steps of 2
- Tip 2: remember that this sum depends on the function `f` that the user ultimately wants to integrate, so it has to take it as an argument additionally to `n`, `a` and `b`

In [7]:
def sum_terms(n, f, a, b):
    """ Calculates the total area of 2^n trapezoids sampled along f(x). 
    
    Args:
        n: algorithm degree
        f: function to be integrated
        a: lower bound of integration
        b: upper bound of integration

    Returns: 
        The result of the sum
    
    """
    summa = 0
    hn = h_n(n, a, b)
    for i in range(1, 2**n, 2):
        summa += f(a + i * hn)
    return summa

#### 3. Write a function `approx_n` that calculates the value of $I_n(a,b)$ for any given function `f`

- Tip 1: make use of `h_n` and `sum_terms` and remember to pass on the arguments
- Tip 2: remember that like $h_n$, $I_n$ is also defined recursively for $n>0$ (eq.1) and has a special definition for the initial estimate $I_0$ (eq. 1)

In [9]:
def approx_n(n, f, a, b, previous_approx=None):
    """ Calculates the n-th order approximation of an integral with the 
    trapezoid method.
    
    Args:
        n: order of the method (i.e. number of trapezoidal "slices")
        f: the function to be integrated. This function f must take a single 
           argument, which is interpreted as the integration variable
        a: lower bound of integration
        b: upper bound of integration
        previous_approx: pre-calculated value of the n-1-th approximation.
                         If not given, all the values will be calculated
                         recursively down to n=0, which is slower

    Returns:
        Apporoximate value for the definite integral of f.
    """
    if n == 0:
        h0 = h_n(0,a,b)
        return (f(a) + f(b)) * h0 / 2
    else:
        if previous_approx is None:                  # This way, the function can
                                                     # be used the old-fashoned
                                                     # (but inefficient) way, by
                                                     # not passing previous_approx
            previous_approx = approx_n(n-1, f, a, b)
        else: 
            previous_approx = previous_approx        # This is the magic line that
                                                     # makes the recursive function 
                                                     # much faster by not computing 
                                                     # I_{n+1} every time
        hn = h_n(n, a, b)
        summa = sum_terms(n, f, a, b)
        return 0.5 * previous_approx + hn * summa


#### 4. Write a function `absepsilon` that takes two different approxiation values and returns the value of $|\varepsilon|$ between them (eq. 3)

- Tip: you can use Python's native abs() function 

In [57]:
def absepsilon(newapprox, oldapprox):
    """Calculate absolute relative difference between two values.
    """
    delta = 1e-200 * (newapprox == 0) # delta = 0 if newapprox != 0, and
                                      # delta = 1e-200 if newapprox == 0,
                                      # so we always avoid dividing by 0.
                                      # This is the same as writing:
                                      # delta = 0 if newapprox else 1e-200
                                      # which is the same as:
                                      # if newapprox: delta = 0
                                      # else: delta = 1e-200
    eps = (newapprox - oldapprox) / (newapprox + delta)
    return abs(eps)

#### 5. Finally, put all the above functions together to create your integrator in the form of a function `trapz(f,a,b, prec)`. Make `prec` a keyword argument so that a precision of `1e-7` is assumed if the user doesn't specify any value. Once the precision is met, the function should return $I_n$, $n$ and $\varepsilon$.

- Tip 1: first, calculate $I_0$. Then starting with $n=1$, calculate a new approximation $I_n$ and compare it with the latest approximation. While your $\varepsilon$ is larger than the desired precision `prec`, keep increasing $n$, replacing the old approximation with the latest one.  

- Tip 2: make your life easier by printing the successive values of $I_n$, $n$ and $\varepsilon$ in every iteration, so you can see what is happening 

In [64]:
def trapz(f, a, b, prec=1e-7):
    """ Calculate the definite integral of f using the trapezoid method.
    
    This is done by successively implementing the trapezoid method to higher
    degrees of accuracy, in a recursive fashion, until the precision prec is
    met.
    
    Args:
        f: the function to be integrated. This function f must take a single 
           argument, which is interpreted as the integration variable
        a: lower bound of integration
        b: upper bound of integration
        prec: minimum precision required. The function keeps increasing the 
              acccuracy degree of the trapezoidal algorithm until the error
              falls below the required precision.

    Returns:
        Best approximation of the integral, number of iterations needed, and 
        error of teh result.
    """
    eps = prec * 10                # Any number larger than prec
    oldapprox = approx_n(0,f,a,b)  # Calculate I_0
    n = 1                          
    while abs(eps) > prec:
        newapprox = approx_n(n,f,a,b,oldapprox) # Bonus: by passing the old 
                                                # approximation, we avoid
                                                # calculating every single
                                                # value recursively
        eps = absepsilon(newapprox, oldapprox)
        oldapprox = newapprox
        n += 1

    return newapprox, n, eps

#### 6. Test your function by computing some integrals:

$
\begin{align}
\int_{-1}^{2} x^3 dx = \left[ \frac{x^4}{4} \right]_{-1}^{2} = 3.75
\end{align}
$


$
\begin{align}
\int_{-a}^{a} (x^5 + x^3) dx = 0 \hspace{5mm} \forall a \in \mathbb{R} \hspace{5mm}\text{(try $a=10^{-5}$ and $a=10^5$)}
\end{align}
$


In [66]:
%%time

f1 = lambda x: x ** 3
f2 = lambda x: x ** 5 + 2 * x ** 3

res1, n1, prec1 = trapz(f1, -1, 2)
res2, n2, prec2 = trapz(f2,-1e-5,1e-5)
res3, n3, prec3 = trapz(f2,-1e5,1e5)


print(f"Integral 1: {res1:.8f} with {n1:2d} iterations and error {prec1:.2e}")
print(f"Integral 2: {res2:.8f} with {n2:2d} iterations and error {prec2:.2e}")
print(f"Integral 3: {res3:.8f} with {n3:2d} iterations and error {prec3:.2e}\n")

Integral 1: 3.75000010 with 14 iterations and error 8.05e-08
Integral 2: 0.00000000 with  2 iterations and error 0.00e+00
Integral 3: 0.00000000 with  2 iterations and error 0.00e+00

CPU times: user 3.35 ms, sys: 171 µs, total: 3.52 ms
Wall time: 3.48 ms
